<a href="https://colab.research.google.com/github/ArtemioPadilla/ArtemioPadilla.github.io/blob/main/URD/DNA/ranking_matrixes_proposals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import re

In [1]:
pip install --upgrade ncbi-datasets-pylib

In [2]:
from ncbi.datasets.openapi import ApiClient as DatasetsApiClient
from ncbi.datasets.openapi import ApiException as DatasetsApiException
from ncbi.datasets import VirusApi as DatasetsVirusApi

from ncbi.datasets.package import dataset

zipfile_name = "sars_cov2_dataset.zip"
linage = "A"

with DatasetsApiClient() as api_client:
    virus_api = DatasetsVirusApi(api_client)
    try:
        print("Begin download of virus data package ...")
        virus_ds_download = virus_api.virus_genome_download(
            "SARS2",
            complete_only=True,
            host="human",
            include_annotation_type=["PROT_FASTA", "CDS_FASTA"],
            _preload_content=False,
            pangolin_classification = linage,
        )

        with open(zipfile_name, "wb") as f:
            f.write(virus_ds_download.data)
        print(f"Download completed -- see {zipfile_name}")
    except DatasetsApiException as e:
        print(f"Exception when calling virus_genome_download: {e}\n")

# open the package zip archive so we can retrieve files from it
package = dataset.VirusDataset(zipfile_name)
# print the names and types of all files in the downloaded zip file
print(package.get_catalog())

Begin download of virus data package ...
Download completed -- see sars_cov2_dataset.zip
{'apiVersion': 'V1', 'assemblies': [{'files': [{'filePath': 'data_report.jsonl', 'fileType': 'DATA_REPORT', 'uncompressedLengthBytes': '14109919'}, {'filePath': 'cds.fna', 'fileType': 'CDS_NUCLEOTIDE_FASTA', 'uncompressedLengthBytes': '34280385'}, {'filePath': 'genomic.fna', 'fileType': 'GENOMIC_NUCLEOTIDE_FASTA', 'uncompressedLengthBytes': '16282453'}, {'filePath': 'protein.faa', 'fileType': 'PROTEIN_FASTA', 'uncompressedLengthBytes': '15825032'}, {'filePath': 'virus_dataset.md', 'fileType': 'README', 'uncompressedLengthBytes': '2431'}]}]}


In [32]:
import json
import jsonlines
import zipfile
import pandas as pd

In [51]:
for g in get_data_reports(zipfile_name):
  print(g.keys())
  break

dict_keys(['accession', 'bioprojects', 'biosample', 'completeness', 'host', 'isolate', 'length', 'location', 'molType', 'nucleotide', 'nucleotideCompleteness', 'releaseDate', 'sourceDatabase', 'sraAccessions', 'updateDate', 'virus'])


In [201]:
def get_data_reports(zip_file):
    with zipfile.ZipFile(zip_file, 'r') as zip_download:
        with zip_download.open('ncbi_dataset/data/data_report.jsonl') as report_file_handle:
            with jsonlines.Reader(report_file_handle) as json_reader:
                for g in json_reader:
                    yield g
   
genome_data = []
for g in get_data_reports(zipfile_name):
    genome_data.append({
        'Accession': g['accession'],
        'TaxID': g['virus']['taxId'],
        'VirusName': g['virus']['sciName'],
        'Host': g.get('host', {}).get('sciName'),
        'Isolate': g.get('isolate', {}).get('name'),
        'Location': g.get('location', {}).get('geographicLocation'),
        'Length': g.get('length', 0),
        'Genes': g.get('geneCount', 0),
        'Proteins': g.get('proteinCount', 0),
        'MaturePeptides': g.get('maturePeptideCount', 0),
        'completeness': g.get('completeness', 0),
        'releaseDate': g.get('releaseDate', 0),
        'updateDate': g.get('updateDate', 0),
        'virus': g.get('virus', None),
    })


df = pd.DataFrame(genome_data)
df = df.sort_values("releaseDate").reset_index(drop=True)
df

,Accession,TaxID,VirusName,Host,Isolate,Location,Length,Genes,Proteins,MaturePeptides,completeness,releaseDate,updateDate,virus
0,MN938384.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,2019-nCoV_HKU-SZ-002a_2020,China: Shenzhen,29838,9,9,0,COMPLETE,2020-01-24,2020-02-11,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
1,MN985325.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/WA-CDC-02982586-001/2020,USA,29882,10,10,0,COMPLETE,2020-01-24,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
2,MN975262.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,2019-nCoV_HKU-SZ-005b_2020,China,29891,10,10,0,COMPLETE,2020-01-24,2020-02-11,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
3,MN988713.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/IL-CDC-02983522-001/2020,USA: Illinois,29882,10,10,0,COMPLETE,2020-01-25,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
4,MN997409.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/AZ-CDC-02993465-001/2020,USA: AZ,29882,10,10,0,COMPLETE,2020-01-28,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,MT318829.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/CHN/ZJNB4794/2020,None,29870,11,12,26,COMPLETE,2022-01-31,2022-01-31,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
532,MT318828.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/CHN/ZJNB016/2020,None,29828,11,12,26,COMPLETE,2022-01-31,2022-01-31,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
533,OM692350.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/KOR/KCDC03-NCCP43326/2021,South Korea,29848,11,12,26,COMPLETE,2022-02-15,2022-02-15,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."
534,OM739099.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/SouthAfrica/NHLS-UCT-GS-2931/...,South Africa,29822,11,12,26,COMPLETE,2022-02-18,2022-02-18,"{'lineage': [{'name': 'Viruses', 'taxId': 1023..."


In [203]:
df["sequence"] = None

In [204]:
pattern_full_gene = r">.*\n(?:[ATNGC]*\n)*"

for genoms_raw, filePath in package.get_files_by_type("GENOMIC_NUCLEOTIDE_FASTA"):
  genoms = re.findall(pattern_full_gene, genoms_raw)

In [205]:
pattern_accession = r">[\w\d\.]*"

pattern_sequence = r"\n(?:[ATGCN]*\n)*"
for genom in genoms:
  accession = re.findall(pattern_accession, genom)[0]
  raw_sequence = re.findall(pattern_sequence, genom)[0]
  sequence = re.sub("\n", "", raw_sequence)
  if accession[1:] == "MT653078.1":
    print(genom)
    print("TEST")
  df.loc[df["Accession"] == accession[1:],"sequence"] = sequence

df.head()

>MT653078.1 Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/BEN/843/2020, complete genome

TEST


,Accession,TaxID,VirusName,Host,Isolate,Location,Length,Genes,Proteins,MaturePeptides,completeness,releaseDate,updateDate,virus,sequence
0,MN938384.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,2019-nCoV_HKU-SZ-002a_2020,China: Shenzhen,29838,9,9,0,COMPLETE,2020-01-24,2020-02-11,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...
1,MN985325.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/WA-CDC-02982586-001/2020,USA,29882,10,10,0,COMPLETE,2020-01-24,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
2,MN975262.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,2019-nCoV_HKU-SZ-005b_2020,China,29891,10,10,0,COMPLETE,2020-01-24,2020-02-11,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
3,MN988713.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/IL-CDC-02983522-001/2020,USA: Illinois,29882,10,10,0,COMPLETE,2020-01-25,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
4,MN997409.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/AZ-CDC-02993465-001/2020,USA: AZ,29882,10,10,0,COMPLETE,2020-01-28,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...


In [215]:
df = df[df.sequence != ""]
df.reset_index(drop=True, inplace=True)
df

,Accession,TaxID,VirusName,Host,Isolate,Location,Length,Genes,Proteins,MaturePeptides,completeness,releaseDate,updateDate,virus,sequence
0,MN938384.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,2019-nCoV_HKU-SZ-002a_2020,China: Shenzhen,29838,9,9,0,COMPLETE,2020-01-24,2020-02-11,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...
1,MN985325.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/WA-CDC-02982586-001/2020,USA,29882,10,10,0,COMPLETE,2020-01-24,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
2,MN975262.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,2019-nCoV_HKU-SZ-005b_2020,China,29891,10,10,0,COMPLETE,2020-01-24,2020-02-11,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
3,MN988713.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/IL-CDC-02983522-001/2020,USA: Illinois,29882,10,10,0,COMPLETE,2020-01-25,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
4,MN997409.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/USA/AZ-CDC-02993465-001/2020,USA: AZ,29882,10,10,0,COMPLETE,2020-01-28,2021-11-08,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,MT318829.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/CHN/ZJNB4794/2020,None,29870,11,12,26,COMPLETE,2022-01-31,2022-01-31,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",TTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGT...
530,MT318828.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/CHN/ZJNB016/2020,None,29828,11,12,26,COMPLETE,2022-01-31,2022-01-31,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",CTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCT...
531,OM692350.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/KOR/KCDC03-NCCP43326/2021,South Korea,29848,11,12,26,COMPLETE,2022-02-15,2022-02-15,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGA...
532,OM739099.1,2697049,Severe acute respiratory syndrome coronavirus 2,Homo sapiens,SARS-CoV-2/human/SouthAfrica/NHLS-UCT-GS-2931/...,South Africa,29822,11,12,26,COMPLETE,2022-02-18,2022-02-18,"{'lineage': [{'name': 'Viruses', 'taxId': 1023...",ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...


# Ranking Functions

In [216]:
def rank_codones(sequence, codon_size = 3):
  freqs = {}

  for i in range(len(sequence)-codon_size+1):
    codon = sequence[i:i+codon_size]

    if codon in freqs:
      freqs[codon] += 1 
    else:
      freqs[codon] = 1 

  return sorted(freqs.items(), key =lambda kv:(kv[1], kv[0]), reverse=True)

In [258]:
rank_codones(df.iloc[0,-1], codon_size = 3)[:10]

[('TTT', 1003),
 ('AAA', 890),
 ('TTA', 873),
 ('TGT', 858),
 ('TTG', 818),
 ('ACA', 807),
 ('ATT', 772),
 ('AAT', 761),
 ('CTT', 737),
 ('ATG', 725)]

In [265]:
def get_ngrams(string, gram_size=3, n_size = 3):
  freqs = {}
  for i in range(1,len(string)-gram_size*n_size, gram_size):
    ngram = string[i:i+gram_size*n_size]
    if ngram in freqs:
      freqs[ngram] += 1 
    else:
      freqs[ngram] = 1 
  return sorted(freqs.items(), key =lambda kv:(kv[1], kv[0]), reverse=True)

get_ngrams(df.iloc[0,-1])[:10]

[('TGGTGTTTA', 5),
 ('TAATGGTGT', 4),
 ('TAAACGAAC', 4),
 ('GTTGATGGT', 4),
 ('TTTTGGTGA', 3),
 ('TTTGTAGTT', 3),
 ('TTTAGATTC', 3),
 ('TTTAATGTT', 3),
 ('TTTAAGGAA', 3),
 ('TTCTTTAAG', 3)]

# Ranking Matrixes

In [270]:

from tqdm.auto import tqdm

def get_rank_matrix(rank_function, df, col):
  """
  Inputs:
  - rank_function: Rank function to get ranking at a given time
  - df: Pandas DataFrame ordered by ascending time
  - col: Column from dataframe to apply rank_function
  Outputs:
  - rank_matrix: Pandas DataFrame with the rank matrix, 
                 each column is a rank at a given time, 
                 columns at the left are older ranks than the ranks in the columns at the right.
  """
  rank_matrix = pd.DataFrame()
  for index, row in tqdm(df.iterrows(), desc="Creating Ranking Matrix", total=len(df)):
    rank = pd.DataFrame.from_dict(rank_function(row[col]))[0] # col 0 is the rank, 1 is the score
    rank_matrix = rank_matrix.append(rank)
  return rank_matrix.T

In [271]:
rank_matrix_codon_freqs = get_rank_matrix(rank_codones, df, "sequence")

rank_matrix_codon_freqs

Creating Ranking Matrix:   0%|          | 0/534 [00:00<?, ?it/s]

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,TTT,TTT,TTT,TCT,TTT,TTT,TTT,TTT,TTT,TTT,...,TTT,TTT,TTT,TTT,TTT,TTT,TTT,TTT,TTT,TTT
1,AAA,AAA,AAA,CTT,AAA,AAA,AAA,AAA,AAA,AAA,...,TTA,TTA,AAA,TTA,AAA,AAA,AAA,AAA,AAA,AAA
2,TTA,TTA,TTA,AAC,TTA,TTA,TTA,TTA,TTA,TTA,...,TGT,TGT,TTA,TGT,TTA,TTA,TTA,TTA,TTA,TTA
3,TGT,TGT,TGT,TGT,TGT,TGT,TGT,TGT,TGT,TGT,...,AAA,AAA,TGT,AAA,TGT,TGT,TGT,TGT,TGT,TGT
4,TTG,TTG,TTG,TAA,TTG,TTG,TTG,TTG,TTG,TTG,...,TTG,TTG,TTG,ACA,TTG,TTG,TTG,TTG,TTG,TTG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
rank_matrix_ngrams = get_rank_matrix(get_ngrams, df, "sequence")

rank_matrix_ngrams

Creating Ranking Matrix:   0%|          | 0/534 [00:00<?, ?it/s]

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,TGGTGTTTA,GGTGTTTAT,GGTGTTTAT,TTTGGAGAC,GGTGTTTAT,GGTGTTTAT,GGTGTTTAT,GGTGTTTAT,GGTGTTTAT,AAAAAAAAA,...,TGTGAAGAA,TTTGTAGTT,GTGTTTATT,GTGTTTATT,GGTGTTTAT,GTGTTTATT,GTGTTTATT,TGGTGTTTA,TGGTGTTTA,TGTTGACAC
1,TAATGGTGT,GTTGTGATG,GTTGTGATG,TTTCGTCCG,GTTGTGATG,GTTGTGATG,GTTGTGATG,GTTGTGATG,GTTGTGATG,GGTGTTTAT,...,GGTGTTGTT,TTCTTTAAG,TGTTGACAC,TGTTGACAC,GTTGTGATG,TGTTGACAC,TGTTGACAC,TAATGGTGT,TAATGGTGT,GTGTTTTAA
2,TAAACGAAC,AATGGTGTT,AATGGTGTT,TTTCGATCT,AATGGTGTT,AATGGTGTT,AATGGTGTT,AATGGTGTT,AATGGTGTT,GTTGTGATG,...,GATGTTGTT,TTATGAAAA,GTGTTTTAA,CTTTTGGTG,AATGGTGTT,GTGTTTTAA,GTGTTTTAA,TAAACGAAC,TAAACGAAC,GTGTTTATT
3,GTTGATGGT,TTTGGTGGT,AAAAAAAAA,TTGCCTGTT,TTTGGTGGT,TTTGGTGGT,TTTGGTGGT,TTTGGTGGT,TTTGGTGGT,AATGGTGTT,...,GAAGCTGCT,TGTAGATGC,CTTTTGGTG,TTGTTAATG,TTTGGTGGT,CTTTTGGTG,CTTTTGGTG,GTTGATGGT,GTTGATGGT,TTTCTGTTT
4,TTTTGGTGA,TTGATGGTG,TTTGGTGGT,TTCTCTAAA,TTGATGGTG,TTGATGGTG,TTGATGGTG,TTGATGGTG,TTGATGGTG,TTTGGTGGT,...,GAAGAAGCT,TGGTGTTTA,TTTCTGTTT,TTAAAGTTA,TTGATGGTG,TTTCTGTTT,TTTCTGTTT,TTTTGGTGA,TTTTGGTGA,TTGTTAATG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Metrics

## Parameters

In [280]:
def get_data_params(df):
    T = df.shape[1] # cada columna es un tiempo
    N0 = df.shape[0] # cada renglon es un elemento del rango
    N = df.stack().nunique() # Elementos únicos en toda la evolución del sistema
    params = {"N":N, "N0":N0, "T":T}

    return params

params = get_data_params(rank_matrix_codon_freqs)
params

{'N': 123, 'N0': 97, 'T': 534}

## Get Properties

In [276]:
def get_matrix_properties(rank_matrix):
  params = get_data_params(rank_matrix)
  print(params)

In [281]:
get_matrix_properties(rank_matrix_codon_freqs)

{'N': 123, 'N0': 97, 'T': 534}


In [282]:
get_matrix_properties(rank_matrix_ngrams)

{'N': 29591, 'N0': 9496, 'T': 534}
